In [1]:
import tensorflow as tf
import numpy as np
import pickle
import os

CIFAR_DIR = './cifar-10-batches-py'
print(os.listdir(CIFAR_DIR))

['data_batch_1', 'readme.html', 'batches.meta', 'data_batch_2', 'data_batch_5', 'test_batch', 'data_batch_4', 'data_batch_3']


In [2]:
def load_data(filename):
    with open(filename, 'rb') as f:
        data = pickle.load(f, encoding='bytes')
        return data[b'data'], data[b'labels']

class CifarData:
    def __init__(self, filenames, need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = load_data(filename)
            all_data.append(data)
            all_labels.append(labels)
        self._data = np.vstack(all_data)
        self._data = self._data / 127.5 - 1  # 为了提高预测准确率，需要对数据进行归一化
        self._labels = np.hstack(all_labels)
        print(self._data.shape)
        print(self._labels.shape)
        
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0  # 当前数据集指明遍历的位置
        if self._need_shuffle:
            self._shuffle_data()
    
    def _shuffle_data(self):
        p = np.random.permutation(self._num_examples)  # 索引进行混排
        self._data = self._data[p]
        self._labels = self._labels[p]
    
    def next_batch(self, batch_size):
        """返回batch_size个样本"""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels

train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1, 6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)

(50000, 3072)
(50000,)
(10000, 3072)
(10000,)


In [3]:
# 计算图模型，先新建计算图，然后将数据塞入计算图中
# placeholder 占位符的作用
x = tf.placeholder(tf.float32, [None, 3072])  # None 样本数不确定，mini_batch
# [None]
y = tf.placeholder(tf.int64, [None])

# get_variable获取变量的函数
# 初始化w
# (3072, 10) 多分类，多神经元
w = tf.get_variable('w', [x.get_shape()[-1], 10],
                   initializer=tf.random_normal_initializer(0, 1))
# 偏置 b
# (10, )
b = tf.get_variable('b', [10],
                  initializer=tf.constant_initializer(0.0))

# 矩阵乘法
# [None, 3072] * [3072, 10] = [None, 10]
y_ = tf.matmul(x, w) + b  # 内积值

# 平方差损失函数
"""
# softmax：e^x / sum(e^x)
# [[0.01, 0.9, ..., 0.03], []]  概率分布，累加为1
p_y = tf.nn.softmax(y_)
# 5 -> [0,0,0,0,1,0,0,0,0,0]
y_one_hot = tf.one_hot(y, 10, dtype=tf.float32)

# 计算损失函数
# reduce_mean 求均值
loss = tf.reduce_mean(tf.square(y_one_hot - p_y))
"""

# 交叉熵损失数函数
loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)
# 执行原理
# y_ -> softmax
# y -> one_hot
# loss -> ylogy_

# 寻找最大概率值的 indices
# 1 代表在第一个维度上
predict = tf.argmax(y_, 1)
# correct_prediction 结果可能是[0,1,0,0,1,1,1]
correct_prediction = tf.equal(predict, y)
# 求准确率，就是求平均值
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

# 在我们有了目标函数和损失函数，再定义梯度下降方法
# 梯度下降的变种
# 优化最小loss的值
with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)
# 计算图构建完成

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [4]:
# 定义 init函数，执行初始化
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 100000
test_steps = 100
# session，打开session后执行计算图
with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, acc_val, _ = sess.run([loss, accuracy, train_op], feed_dict={x: batch_data, y: batch_labels})
        if (i+1) % 500 == 0:
            print('[Train] Step: %d, loss: %4.5f, acc: %4.5f' % (i+1, loss_val, acc_val))
        if (i+1) % 5000 == 0:
            test_data = CifarData(test_filenames, False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels = test_data.next_batch(batch_size)
                test_acc_val = sess.run([accuracy], feed_dict={x: test_batch_data, y: test_batch_labels})
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[Test ] Step: %d, acc: %4.5f' % (i+1, test_acc))

[Train] Step: 500, loss: 15.27979, acc: 0.25000
[Train] Step: 1000, loss: 14.69378, acc: 0.15000
[Train] Step: 1500, loss: 13.40500, acc: 0.30000
[Train] Step: 2000, loss: 11.23500, acc: 0.20000
[Train] Step: 2500, loss: 7.48543, acc: 0.30000
[Train] Step: 3000, loss: 10.68519, acc: 0.15000
[Train] Step: 3500, loss: 8.09123, acc: 0.25000
[Train] Step: 4000, loss: 7.64369, acc: 0.25000
[Train] Step: 4500, loss: 10.94256, acc: 0.25000
[Train] Step: 5000, loss: 7.89266, acc: 0.25000
(10000, 3072)
(10000,)
[Test ] Step: 5000, acc: 0.25500
[Train] Step: 5500, loss: 10.22781, acc: 0.10000
[Train] Step: 6000, loss: 6.74785, acc: 0.35000
[Train] Step: 6500, loss: 7.56665, acc: 0.30000
[Train] Step: 7000, loss: 7.51485, acc: 0.25000
[Train] Step: 7500, loss: 8.64874, acc: 0.35000
[Train] Step: 8000, loss: 5.57807, acc: 0.25000
[Train] Step: 8500, loss: 10.72903, acc: 0.10000
[Train] Step: 9000, loss: 9.36287, acc: 0.15000
[Train] Step: 9500, loss: 6.96482, acc: 0.10000
[Train] Step: 10000, loss

[Train] Step: 76000, loss: 2.85096, acc: 0.25000
[Train] Step: 76500, loss: 3.15100, acc: 0.15000
[Train] Step: 77000, loss: 2.63354, acc: 0.40000
[Train] Step: 77500, loss: 2.04440, acc: 0.45000
[Train] Step: 78000, loss: 2.74822, acc: 0.30000
[Train] Step: 78500, loss: 2.36136, acc: 0.35000
[Train] Step: 79000, loss: 2.58961, acc: 0.30000
[Train] Step: 79500, loss: 2.76062, acc: 0.35000
[Train] Step: 80000, loss: 2.14596, acc: 0.45000
(10000, 3072)
(10000,)
[Test ] Step: 80000, acc: 0.30650
[Train] Step: 80500, loss: 2.61863, acc: 0.35000
[Train] Step: 81000, loss: 1.92212, acc: 0.45000
[Train] Step: 81500, loss: 2.07143, acc: 0.35000
[Train] Step: 82000, loss: 2.75772, acc: 0.35000
[Train] Step: 82500, loss: 2.92808, acc: 0.20000
[Train] Step: 83000, loss: 2.07761, acc: 0.40000
[Train] Step: 83500, loss: 2.41449, acc: 0.40000
[Train] Step: 84000, loss: 2.45515, acc: 0.15000
[Train] Step: 84500, loss: 2.56543, acc: 0.35000
[Train] Step: 85000, loss: 2.37418, acc: 0.35000
(10000, 3072